In order to use the Conch encoder, follow the steps here: https://huggingface.co/MahmoodLab/CONCH

The model runs pretty fast even on CPU. Haven't tested on GPU yet because, as always, cuda is a pain to install and setup (and im not able to install the conch library properly with conda which wouldve made using cuda easy).

In [8]:
import os
from dotenv import load_dotenv

load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")

In [2]:
from conch.open_clip_custom import create_model_from_pretrained
model, preprocess = create_model_from_pretrained('conch_ViT-B-16', "hf_hub:MahmoodLab/conch", hf_auth_token=HF_TOKEN)

C:\Users\justaguy\Desktop\Concordia University\Fall24\COMP433\Huron-Tissue-Segment\.venv\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
C:\Users\justaguy\Desktop\Concordia University\Fall24\COMP433\Huron-Tissue-Segment\.venv\lib\site-packages\conch\open_clip_custom\factory.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary obje

In [4]:
import torch
from PIL import Image
image = Image.open("data/Huron_data/Sliced_Images/1.png")
image = preprocess(image).unsqueeze(0)
with torch.inference_mode():
    image_embs = model.encode_image(image, proj_contrast=False, normalize=False)

In [6]:
image_embs, image_embs.shape

(tensor([[-9.0573e-01,  6.0244e-01,  2.5481e-01,  3.6276e-01,  6.7336e-01,
           1.1095e+00, -1.5306e+00,  2.2528e-01, -1.2526e+00, -1.2078e-02,
           2.2109e-01,  1.0750e-01,  6.2914e-01, -1.9161e-01,  1.6931e-01,
           2.3996e+00,  1.4053e+00,  7.9867e-02,  7.6496e-01, -1.5338e+00,
          -8.3441e-01, -4.8194e-01, -1.5833e+00,  1.3520e-01, -1.0544e-01,
           1.5729e+00,  3.0416e-01, -1.4083e+00,  1.2311e+00,  1.2755e-01,
          -1.5042e+00, -9.9686e-01,  2.5562e-01,  6.5349e-01, -3.1810e-01,
          -2.7508e-01, -1.4575e+00, -1.1774e-01,  1.3684e-02,  1.4021e-01,
          -1.4756e-01, -1.9825e-01,  1.1501e+00,  1.1381e+00,  1.1352e-01,
          -3.1522e-01,  8.0569e-01,  1.0948e+00,  2.7276e-01,  5.1390e-01,
           3.4829e-01, -3.6323e-01, -7.4146e-01, -9.8186e-01, -2.1282e-01,
           9.2595e-01, -9.2716e-01,  4.2866e-01,  8.6721e-02,  1.6750e-01,
          -1.1661e+00,  2.1819e+00,  7.6310e-01, -1.5981e-01,  1.5913e-01,
           2.9533e-01,  1

## Example function for converting images to embeddings

In [13]:
import cv2 # cv2 loads images faster from what ive seen than PIL
from PIL import Image

def image_to_embedding(model, image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    image = preprocess(image).unsqueeze(0)
    with torch.inference_mode():
        image_embs = model.encode_image(image, proj_contrast=False, normalize=False)
    return image_embs

In [14]:
image_to_embedding(model, "data/Huron_data/Sliced_Images/1.png")

tensor([[-9.0573e-01,  6.0244e-01,  2.5481e-01,  3.6276e-01,  6.7336e-01,
          1.1095e+00, -1.5306e+00,  2.2528e-01, -1.2526e+00, -1.2078e-02,
          2.2109e-01,  1.0750e-01,  6.2914e-01, -1.9161e-01,  1.6931e-01,
          2.3996e+00,  1.4053e+00,  7.9867e-02,  7.6496e-01, -1.5338e+00,
         -8.3441e-01, -4.8194e-01, -1.5833e+00,  1.3520e-01, -1.0544e-01,
          1.5729e+00,  3.0416e-01, -1.4083e+00,  1.2311e+00,  1.2755e-01,
         -1.5042e+00, -9.9686e-01,  2.5562e-01,  6.5349e-01, -3.1810e-01,
         -2.7508e-01, -1.4575e+00, -1.1774e-01,  1.3684e-02,  1.4021e-01,
         -1.4756e-01, -1.9825e-01,  1.1501e+00,  1.1381e+00,  1.1352e-01,
         -3.1522e-01,  8.0569e-01,  1.0948e+00,  2.7276e-01,  5.1390e-01,
          3.4829e-01, -3.6323e-01, -7.4146e-01, -9.8186e-01, -2.1282e-01,
          9.2595e-01, -9.2716e-01,  4.2866e-01,  8.6721e-02,  1.6750e-01,
         -1.1661e+00,  2.1819e+00,  7.6310e-01, -1.5981e-01,  1.5913e-01,
          2.9533e-01,  1.9766e-01, -3.